In [2]:
# This notebook illustrates handling the August 2024 Demo of the 6mo Hackathon Scenario 2, described at: https://github.com/DARPA-ASKEM/program-milestones/issues/74

# Import funman related code
import os
from funman.api.run import Runner
from funman_demo import summarize_results
from funman import FunmanWorkRequest, EncodingSchedule 
import json
from funman.representation.constraint import LinearConstraint, ParameterConstraint, StateVariableConstraint
from funman.representation import Interval
import pandas as pd

RESOURCES = "../../resources"
SAVED_RESULTS_DIR = "./out"

EXAMPLE_DIR = os.path.join(RESOURCES, "amr", "petrinet","monthly-demo", "2024-08", "6_month_scenario_2", "q1b", "part_1")
MODEL_PATH = os.path.join(
    EXAMPLE_DIR, "BIOMD0000000955_askenet.json"
)
REQUEST_PATH = os.path.join(
    EXAMPLE_DIR, "BIOMD0000000955_askenet_request.json"
)

request_params = {}

# %load_ext autoreload
# %autoreload 2

In [3]:
# Constants for the scenario
STATES = ["Susceptible", "Diagnosed", "Infected", "Ailing", "Recognized", "Healed", "Threatened", "Extinct"]
IDART = ["Diagnosed", "Infected", "Ailing", "Recognized",  "Threatened"]

MAX_TIME=50
STEP_SIZE=2
timepoints = list(range(0, MAX_TIME+STEP_SIZE, STEP_SIZE))

In [4]:
# Helper functions to setup FUNMAN for different steps of the scenario

def get_request():
    with open(REQUEST_PATH, "r") as request:
        funman_request = FunmanWorkRequest.model_validate(json.load(request))
        return funman_request

def set_timepoints(funman_request, timepoints):
    funman_request.structure_parameters[0].schedules = [EncodingSchedule(timepoints=timepoints)]

def unset_all_labels(funman_request):
    for p in funman_request.parameters:
        p.label = "any"
    
def set_config_options(funman_request, debug=False, dreal_precision=1e-1):
    # Overrides for configuration
    #
    # funman_request.config.substitute_subformulas = True
    # funman_request.config.use_transition_symbols = True
    # funman_request.config.use_compartmental_constraints=False
    if debug:
        funman_request.config.save_smtlib="./out"
    funman_request.config.tolerance = 0.01
    funman_request.config.dreal_precision = dreal_precision
    # funman_request.config.verbosity = 10
    # funman_request.config.dreal_log_level = "debug"
    # funman_request.config.dreal_prefer_parameters = ["beta","NPI_mult","r_Sv","r_EI","r_IH_u","r_IH_v","r_HR","r_HD","r_IR_u","r_IR_v"]

def get_synthesized_vars(funman_request):
    return [p.name for p in funman_request.parameters if p.label == "all"]

def run(funman_request, plot=False):
    to_synthesize = get_synthesized_vars(funman_request)
    return Runner().run(
        MODEL_PATH,
        funman_request,
        description="SIDARTHE Eval Scenario 2.1.b.1 6mo",
        case_out_dir=SAVED_RESULTS_DIR,
        dump_plot=plot,
        print_last_time=True,
        parameters_to_plot=to_synthesize
    )

def setup_common(funman_request, synthesize=False, debug=False, dreal_precision=1e-1):
    set_timepoints(funman_request, timepoints)
    if not synthesize:
        unset_all_labels(funman_request)
    set_config_options(funman_request, debug=debug, dreal_precision=dreal_precision)
    

def set_compartment_bounds(funman_request, upper_bound=9830000.0, error=0.01):
    # Add bounds to compartments
    for var in STATES:
        funman_request.constraints.append(StateVariableConstraint(name=f"{var}_bounds", variable=var, interval=Interval(lb=0, ub=upper_bound, closed_upper_bound=True),soft=False))

    # Add sum of compartments
    funman_request.constraints.append(LinearConstraint(name=f"compartment_bounds", variables=STATES, additive_bounds=Interval(lb=upper_bound-error, ub=upper_bound+error, closed_upper_bound=False), soft=True))

def relax_parameter_bounds(funman_request, factor = 0.1):
    # Relax parameter bounds
    parameters = funman_request.parameters
    for p in parameters:
        interval = p.interval
        width = float(interval.width())
        interval.lb = interval.lb - (factor/2 * width)
        interval.ub = interval.ub + (factor/2 * width)

def plot_last_point(results):
    pts = results.parameter_space.points() 
    print(f"{len(pts)} points")

    if len(pts) > 0:
        # Get a plot for last point
        df = results.dataframe(points=pts[-1:])
        ax = df[STATES].plot()
        ax.set_yscale("log")

def get_last_point_parameters(results):
    pts = results.parameter_space.points()
    if len(pts) > 0:
        pt = pts[-1]
        parameters = results.model._parameter_names()
        param_values = {k:v for k, v in pt.values.items() if k in parameters }
        return param_values

def pretty_print_request_params(params):
    # print(json.dump(params, indent=4))
    if len(params)>0:

        df = pd.DataFrame(params)
        print(df.T)


def report(results, name):
    plot_last_point(results)
    param_values = get_last_point_parameters(results)
    # print(f"Point parameters: {param_values}")
    if param_values is not None:
        request_params[name] = param_values
    pretty_print_request_params(request_params)

def add_unit_test(funman_request):
    funman_request.constraints.append(LinearConstraint(name="unit_test", variables = [
            "Infected",
            "Diagnosed",
            "Ailing",
            "Recognized",
            "Threatened"
         ],
         additive_bounds= {
            "lb": 0.55,
            "ub": 0.65
         },
         timepoints={
            "lb": 45,
            "ub": 55
         }
      ))


In [6]:
# Find a single parameterization of the model where sum(IDART) is approx 60% around day 47.

funman_request = get_request()
setup_common(funman_request, debug=True, dreal_precision=1e0)
add_unit_test(funman_request)
results = run(funman_request)
# report(results, "unconstrained")

2024-08-16 18:17:23,491 - funman.server.worker - INFO - FunmanWorker running...
2024-08-16 18:17:23,495 - funman.server.worker - INFO - Starting work on: 81205490-5e77-4321-bbd4-d38e2c4828f0
2024-08-16 18:17:31,582 - funman.api.run - INFO - Dumping results to ./out/81205490-5e77-4321-bbd4-d38e2c4828f0.json
2024-08-16 18:17:31,632 - funman.scenario.consistency - INFO - 25{50}:	[+]
2024-08-16 18:17:31,679 - funman.server.worker - INFO - Completed work on: 81205490-5e77-4321-bbd4-d38e2c4828f0
2024-08-16 18:17:41,625 - funman.server.worker - INFO - Worker.stop() acquiring state lock ....
2024-08-16 18:17:41,754 - funman.server.worker - INFO - FunmanWorker exiting...
2024-08-16 18:17:41,757 - funman.server.worker - INFO - Worker.stop() completed.


In [ ]:
results
# get_last_point_parameters(results)
# plot_last_point(results)
# def plot_last_point(results):
#     pts = results.parameter_space.points() 
#     print(f"{len(pts)} points")

#     if len(pts) > 0:
#         # Get a plot for last point
#         df = results.dataframe(points=pts[-1:])
#         ax = df[STATES].plot()
#         ax.set_yscale("log")
pts = results.parameter_space.points() 
print(f"{len(pts)} points")
df = results.dataframe(points=pts[-1:])
# df[IDART].loc(df.index==50.0)
IDART_by_day = df[IDART].sum(axis=1)
error = 0.02
diff_from_target = abs(IDART_by_day - 0.60)
close_to_target = any(diff_from_target < error)
close_to_target
# IDART_by_day
# abs(day45_IDART - 47)
# df[IDART].sum(axis=1).plot()

In [ ]:
import matplotlib.pyplot as plt

# plt.plot(df.Infected)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame({"a": [0, 1], "b": [3, 4]})
l = list(df.a.values)
# l = [float(v) for v in l]
type(l[0])

# p = plt.plot(df.a, df.b)
df
# print(p)
# plt.plot(l)

# plt.plot(df.a.values)
# plt.plot(df['a'])
# df.plot()

In [ ]:
# Add bounds [0, N] to the STATE compartments.  
# Add bounds sum(STATE) in [N-e, N+e], for a small e.

funman_request = get_request()
setup_common(funman_request, debug=True)
set_compartment_bounds(funman_request)
results = run(funman_request)
report(results, "compartmental_constrained")

In [ ]:
# Relax the bounds on the parameters to allow additional parameterizations

funman_request = get_request()
setup_common(funman_request)
set_compartment_bounds(funman_request)
relax_parameter_bounds(funman_request, factor = 0.75)
results = run(funman_request)
report(results, "relaxed_bounds")

In [ ]:
funman_request = get_request()
setup_common(funman_request, synthesize=True)
set_compartment_bounds(funman_request)
# relax_parameter_bounds(funman_request, factor=0.75)
# funman_request.config.verbosity=10
results = run(funman_request, plot=True)
report(results, "synthesis")

In [ ]:
# import pandas as pd
# df1 = pd.DataFrame({"ltp": ltp, "gtp": gtp})

# # df2 = 
# # df1.ltp.N == df1.gtp.N
# # df2.loc[df2].sort_index()[0:60]

# #(= H_10 (+ H_8 (* 2 (+ (* r_HD (- 1) H_8) (* r_HR (- 1) H_8) (* r_IH_v I_v_8) (* r_IH_u I_u_8)))))

# df1['same'] = df1['ltp'] == df1["gtp"]
# df1[0:20]
# # df1.loc[df1.index.str.endswith("_6")].sort_values(by="same")

In [ ]:
# # Get points (trajectories generated)
# pts = results.parameter_space.points() 
# print(f"{len(pts)} points")

# # Get a plot for last point
# df = results.dataframe(points=pts[-1:])
# ax = df[STATES].plot()
# ax.set_yscale("log")


# # Get the values of the point
# gtp=pts[-1].values


# # Output the model diagram
# #
results.model.to_dot()
# # gtp